In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [6]:
# set data path
train_path = "iris_training.csv"
test_path = "iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']

In [10]:
def load_data(train_path, test_path, y_name='Species'):
    # Return the iris dataset as (train_x, train_y), (test_x, test_y)
    
    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)
    
    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)
    
    return (train_x, train_y), (test_x, test_y)

# load data
(train_x, train_y), (test_x, test_y) = load_data(train_path, test_path)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(120, 4)
(120,)
(30, 4)
(30,)


In [11]:
learning_rate = 0.001
num_epochs = 100
display_step = 1 # 손실함수 출력 주기
input_size = 4 # SepalLength, SepalWidth, PetalLength, PetalWidth
hidden1_size = 256
hidden2_size = 256
output_size = 3 # Setosa, Versicolor, Virginica

x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, output_size])

In [12]:
def build_ANN(x):
    W1 = tf.Variable(tf.random_normal(shape=[input_size, hidden1_size]))
    b1 = tf.Variable(tf.random_normal(shape=[hidden1_size]))
    H1_output = tf.nn.relu(tf.matmul(x, W1) + b1)
    
    W2 = tf.Variable(tf.random_normal(shape=[hidden1_size, hidden2_size]))
    b2 = tf.Variable(tf.random_normal(shape=[hidden2_size]))
    H2_output = tf.nn.relu(tf.matmul(H1_output, W2) + b2)
    
    W_output = tf.Variable(tf.random_normal(shape=[hidden2_size, output_size]))
    b_output = tf.Variable(tf.random_normal(shape=[output_size]))
    logits = tf.matmul(H2_output, W_output) + b_output
    return logits

In [13]:
# ANN모델 선언
predicted_value = build_ANN(x)

# 손실함수, 옵티마이저 정의
# tf.nn.softmax_cross_entropy_with_logits 함수를 이용하여 활성함수를 적용하지 않은 output layer의
# 결과값(logits)에 softmax 함수를 적용
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_value, labels=y))
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [16]:
# 세션을 열고 그래프 실행
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 지정된 횟수만큼 최적화
    for epoch in range(num_epochs):
        average_loss = 0.
        
        batch_x = train_x.values
        batch_y = sess.run(tf.one_hot(train_y.values, 3))
        # 옵티마이저 실행, 파라미터 업데이트
        _, current_loss = sess.run([train_step, loss], feed_dict={x: batch_x, y: batch_y})
        
        # 평균 손실 측정
        average_loss += current_loss / batch_x.shape[0]
            
        #지정된 epoch마다 학습결과 출력
        if epoch % display_step == 0:
            print("반복(Epoch): %d, 손실함수(Loss): %f" % ((epoch+1), average_loss))
            
    # 정확도 출력
    correct_prediction = tf.equal(tf.argmax(predicted_value, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    print("정확도(Accuracy): %f" % (accuracy.eval(feed_dict={x: test_x.values, y: sess.run(tf.one_hot(test_y.values, 3))})))

반복(Epoch): 1, 손실함수(Loss): 4.867050
반복(Epoch): 2, 손실함수(Loss): 4.234914
반복(Epoch): 3, 손실함수(Loss): 3.602315
반복(Epoch): 4, 손실함수(Loss): 3.002183
반복(Epoch): 5, 손실함수(Loss): 2.543899
반복(Epoch): 6, 손실함수(Loss): 2.218004
반복(Epoch): 7, 손실함수(Loss): 1.948457
반복(Epoch): 8, 손실함수(Loss): 1.776784
반복(Epoch): 9, 손실함수(Loss): 1.636147
반복(Epoch): 10, 손실함수(Loss): 1.440999
반복(Epoch): 11, 손실함수(Loss): 1.181122
반복(Epoch): 12, 손실함수(Loss): 0.892328
반복(Epoch): 13, 손실함수(Loss): 0.605661
반복(Epoch): 14, 손실함수(Loss): 0.343097
반복(Epoch): 15, 손실함수(Loss): 0.127870
반복(Epoch): 16, 손실함수(Loss): 0.088958
반복(Epoch): 17, 손실함수(Loss): 0.185171
반복(Epoch): 18, 손실함수(Loss): 0.307292
반복(Epoch): 19, 손실함수(Loss): 0.372370
반복(Epoch): 20, 손실함수(Loss): 0.321172
반복(Epoch): 21, 손실함수(Loss): 0.198013
반복(Epoch): 22, 손실함수(Loss): 0.084958
반복(Epoch): 23, 손실함수(Loss): 0.025054
반복(Epoch): 24, 손실함수(Loss): 0.014070
반복(Epoch): 25, 손실함수(Loss): 0.018052
반복(Epoch): 26, 손실함수(Loss): 0.031091
반복(Epoch): 27, 손실함수(Loss): 0.045072
반복(Epoch): 28, 손실함수(Loss): 0.055999
반